In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ls

drive/  sample_data/


In [3]:
cd drive/

/content/drive


In [4]:
cd MyDrive

/content/drive/MyDrive


In [5]:
cd 709/Ass2

/content/drive/MyDrive/709/Ass2


In [6]:
ls

 gold1.csv/          gold.csv   spark-warehouse/  ' Task 2.ipynb'  'Task3 test 1.ipynb'
 gold1_single.csv/   models/   'Task 1.ipynb'     'Task 3.ipynb'    Untitled0.ipynb


In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [8]:
df = pd.read_csv('gold.csv')

In [9]:
df.shape

(5703, 7)

In [10]:
df.head()

,Date,Open,High,Low,Close,Volume,Currency
0,2000-01-04,289.5,289.5,280.0,283.7,21621,USD
1,2000-01-05,283.7,285.0,281.0,282.1,25448,USD
2,2000-01-06,281.6,282.8,280.2,282.4,19055,USD
3,2000-01-07,282.5,284.5,282.0,282.9,11266,USD
4,2000-01-10,282.4,283.9,281.8,282.7,30603,USD


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5703 entries, 0 to 5702
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      5703 non-null   object 
 1   Open      5703 non-null   float64
 2   High      5703 non-null   float64
 3   Low       5703 non-null   float64
 4   Close     5703 non-null   float64
 5   Volume    5703 non-null   int64  
 6   Currency  5703 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 312.0+ KB


In [12]:
df.isnull().sum()

,0
Date,0
Open,0
High,0
Low,0
Close,0
Volume,0
Currency,0


In [13]:
df.describe(include = 'all')

,Date,Open,High,Low,Close,Volume,Currency
count,5703,5703.000000,5703.000000,5703.000000,5703.000000,5703.000000,5703
unique,5703,NaN,NaN,NaN,NaN,NaN,1
top,2000-01-04,NaN,NaN,NaN,NaN,NaN,USD
freq,1,NaN,NaN,NaN,NaN,NaN,5703
mean,NaN,1040.382816,1048.339181,1031.863169,1040.298282,139141.669297,NaN
std,NaN,518.733377,522.353946,514.455903,518.524020,102537.449058,NaN
min,NaN,256.600000,259.400000,255.100000,256.600000,0.000000,NaN
25%,NaN,459.850000,463.900000,457.450000,460.500000,52938.500000,NaN
50%,NaN,1188.800000,1198.000000,1179.700000,1188.700000,126006.000000,NaN
75%,NaN,1381.400000,1392.750000,1368.100000,1383.050000,193109.000000,NaN


In [14]:
# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Find the earliest and latest dates
earliest_date = df['Date'].min()
latest_date = df['Date'].max()

print(f"Earliest Date: {earliest_date}")
print(f"Latest Date: {latest_date}")

Earliest Date: 2000-01-04 00:00:00
Latest Date: 2022-09-02 00:00:00


In [15]:
# Remove rows where the 'Date' is '2022-09'
df = df[df['Date'].dt.strftime('%Y-%m') != '2022-09']

In [16]:
df.shape

(5701, 7)

In [17]:
df.sort_values('Date', inplace=True)

# Calculate the change percentage
df['Change%'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)) * 100

# Round the 'Change%' column to 2 decimal places
df['Change%'] = df['Change%'].round(2)
df['Change%'] = df['Change%'].fillna(0)

In [18]:
# Save the amended DataFrame to a new CSV file
df.to_csv('gold1.csv', index=False)

IsADirectoryError: [Errno 21] Is a directory: 'gold1.csv'

In [19]:
df.head()

,Date,Open,High,Low,Close,Volume,Currency,Change%
0,2000-01-04,289.5,289.5,280.0,283.7,21621,USD,0.00
1,2000-01-05,283.7,285.0,281.0,282.1,25448,USD,-0.56
2,2000-01-06,281.6,282.8,280.2,282.4,19055,USD,0.11
3,2000-01-07,282.5,284.5,282.0,282.9,11266,USD,0.18
4,2000-01-10,282.4,283.9,281.8,282.7,30603,USD,-0.07


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import to_date
from pyspark.sql.functions import year, month
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql.functions import col, lag, round
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [21]:
!pip install hdfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34324 sha256=f89bd7c1540aa3a4998ab9904345241034aaf2e916ba12f3b38e7866a5080537
  Stored in directory: /root/.cache/pip/wheels/b9/1d/dc/eb0833be25464c359903d356c4204721c6a672c26ff164cdc3
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=857fbbf58de5f3fc4778f4489d54cb7fe94ba40aaac4109b52e0b399d620488b
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built hdfs docopt


In [22]:
!pip install pyspark

In [23]:
spark = SparkSession.builder.appName("PySpark Example").getOrCreate()

df = spark.read.csv('gold.csv', header=True, inferSchema=True)
df.show()

+----------+-----+-----+-----+-----+------+--------+
|      Date| Open| High|  Low|Close|Volume|Currency|
+----------+-----+-----+-----+-----+------+--------+
|2000-01-04|289.5|289.5|280.0|283.7| 21621|     USD|
|2000-01-05|283.7|285.0|281.0|282.1| 25448|     USD|
|2000-01-06|281.6|282.8|280.2|282.4| 19055|     USD|
|2000-01-07|282.5|284.5|282.0|282.9| 11266|     USD|
|2000-01-10|282.4|283.9|281.8|282.7| 30603|     USD|
|2000-01-11|282.4|285.3|281.9|284.4| 13500|     USD|
|2000-01-12|284.5|285.0|282.5|283.7| 17841|     USD|
|2000-01-13|283.5|285.9|282.9|285.1| 12171|     USD|
|2000-01-14|285.2|285.6|284.0|284.9| 32339|     USD|
|2000-01-18|286.1|289.8|285.9|289.6| 28615|     USD|
|2000-01-19|289.6|291.0|287.5|290.3| 24097|     USD|
|2000-01-20|290.3|290.7|288.4|289.3| 20707|     USD|
|2000-01-21|289.0|290.3|287.4|289.7| 25726|     USD|
|2000-01-24|289.7|289.7|286.3|288.1| 32140|     USD|
|2000-01-25|288.0|291.8|286.6|286.6| 36122|     USD|
|2000-01-26|286.8|287.5|284.7|286.5| 19633|   

In [24]:
df.describe().show()

+-------+------------------+------------------+------------------+-----------------+------------------+--------+
|summary|              Open|              High|               Low|            Close|            Volume|Currency|
+-------+------------------+------------------+------------------+-----------------+------------------+--------+
|  count|              5703|              5703|              5703|             5703|              5703|    5703|
|   mean|1040.3828160617231|1048.3391811327342|1031.8631685078012|1040.298281606174| 139141.6692968613|    NULL|
| stddev|  518.733376764074| 522.3539457427499|  514.455902840454|518.5240202194819|102537.44905787897|    NULL|
|    min|             256.6|             259.4|             255.1|            256.6|                 0|     USD|
|    max|            2076.4|            2089.2|            2049.0|           2069.4|            816531|     USD|
+-------+------------------+------------------+------------------+-----------------+------------

In [25]:
df.select([countDistinct(c).alias(c) for c in df.columns]).show()

+----+----+----+----+-----+------+--------+
|Date|Open|High| Low|Close|Volume|Currency|
+----+----+----+----+-----+------+--------+
|5703|4364|4414|4375| 4445|  5594|       1|
+----+----+----+----+-----+------+--------+



In [26]:
rows = df.count()
cols = len(df.columns)
print(f"The DataFrame has {rows} rows and {cols} columns.")

The DataFrame has 5703 rows and 7 columns.


In [27]:
df.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+----+----+---+-----+------+--------+
|Date|Open|High|Low|Close|Volume|Currency|
+----+----+----+---+-----+------+--------+
|   0|   0|   0|  0|    0|     0|       0|
+----+----+----+---+-----+------+--------+



In [28]:
df = df.withColumn("Date", to_date(col("Date"), 'yyyy-MM-dd'))
df.select(max("date")).show()
df.select(min("date")).show()

+----------+
| max(date)|
+----------+
|2022-09-02|
+----------+

+----------+
| min(date)|
+----------+
|2000-01-04|
+----------+



In [29]:
df =df.filter((year(col("Date")) != 2022) | (month(col("Date")) != 9))

In [30]:
rows = df.count()
cols = len(df.columns)
print(f"The DataFrame has {rows} rows and {cols} columns.")

The DataFrame has 5701 rows and 7 columns.


In [31]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Currency: string (nullable = true)



In [32]:
df.groupBy(year("Date").alias("year"), month("Date").alias("month")).count().orderBy("year", "month").show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2000|    1|   19|
|2000|    2|   20|
|2000|    3|   23|
|2000|    4|   19|
|2000|    5|   22|
|2000|    6|   22|
|2000|    7|   19|
|2000|    8|   23|
|2000|    9|   20|
|2000|   10|   22|
|2000|   11|   20|
|2000|   12|   20|
|2001|    1|   21|
|2001|    2|   19|
|2001|    3|   22|
|2001|    4|   20|
|2001|    5|   22|
|2001|    6|   21|
|2001|    7|   21|
|2001|    8|   23|
+----+-----+-----+
only showing top 20 rows



In [33]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define a window specification ordered by Date
window_spec = Window.orderBy("Date")

# Calculate Price Range (High - Low)
df = df.withColumn("PriceRange", round(col("High") - col("Low"), 2))

# Calculate Log Return and Daily Change % together
df = df.withColumn(
    "LogReturn",
    F.round(F.log(F.col("Close") / F.lag("Close").over(window_spec)), 5)
).withColumn(
    "DailyChange%",
    F.round(((F.col("Close") - F.lag("Close").over(window_spec)) / F.lag("Close").over(window_spec)) * 100, 2)
)

# Calculate Short-Term Moving Average (e.g., 30 days)
short_window_spec = Window.orderBy("Date").rowsBetween(-29, 0)  # Last 30 days
df = df.withColumn("ShortMA", round(avg(col("Close")).over(short_window_spec), 2))

# Calculate Long-Term Moving Average (e.g., 120 days)
long_window_spec = Window.orderBy("Date").rowsBetween(-119, 0)  # Last 120 days
df = df.withColumn("LongMA", round(avg(col("Close")).over(long_window_spec), 2))

# Calculate Volatility (Rolling Standard Deviation)
df = df.withColumn("Volatility", round(stddev(col("Close")).over(long_window_spec), 2))

# Save the resulting DataFrame to a new CSV file
df.write.csv('gold1.csv', header=True, mode='overwrite')

print("File saved as 'gold1.csv' with added features: PriceRange, LogReturn, DailyChange%, ShortMA, LongMA, Volatility.")

File saved as 'gold1.csv' with added features: PriceRange, LogReturn, DailyChange%, ShortMA, LongMA, Volatility.


In [34]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Currency: string (nullable = true)
 |-- PriceRange: double (nullable = true)
 |-- LogReturn: double (nullable = true)
 |-- DailyChange%: double (nullable = true)
 |-- ShortMA: double (nullable = true)
 |-- LongMA: double (nullable = true)
 |-- Volatility: double (nullable = true)



In [35]:
df = df.fillna(0)

In [36]:
# Extract day of the week from the Date column
df = df.withColumn("DayOfWeek", F.date_format(F.col("Date"), "EEEE"))

In [37]:
df = df.drop("Currency")
# Save the resulting DataFrame to a new CSV file
df.write.csv('gold1.csv', header=True, mode='overwrite')

In [38]:
df.show(5)

+----------+-----+-----+-----+-----+------+----------+---------+------------+-------+------+----------+---------+
|      Date| Open| High|  Low|Close|Volume|PriceRange|LogReturn|DailyChange%|ShortMA|LongMA|Volatility|DayOfWeek|
+----------+-----+-----+-----+-----+------+----------+---------+------------+-------+------+----------+---------+
|2000-01-04|289.5|289.5|280.0|283.7| 21621|       9.5|      0.0|         0.0|  283.7| 283.7|       0.0|  Tuesday|
|2000-01-05|283.7|285.0|281.0|282.1| 25448|       4.0| -0.00566|       -0.56|  282.9| 282.9|      1.13|Wednesday|
|2000-01-06|281.6|282.8|280.2|282.4| 19055|       2.6|  0.00106|        0.11| 282.73|282.73|      0.85| Thursday|
|2000-01-07|282.5|284.5|282.0|282.9| 11266|       2.5|  0.00177|        0.18| 282.78|282.78|       0.7|   Friday|
|2000-01-10|282.4|283.9|281.8|282.7| 30603|       2.1|  -7.1E-4|       -0.07| 282.76|282.76|      0.61|   Monday|
+----------+-----+-----+-----+-----+------+----------+---------+------------+-------+---

In [39]:
# Repartition the DataFrame to 1 partition to write a single file
df.coalesce(1).write.csv('gold1_single.csv', header=True, mode='overwrite')

# Check the directory to see the result
!ls gold1_single.csv


part-00000-d6bcc627-108d-4fc3-a854-516748bfec5e-c000.csv  _SUCCESS


In [40]:
import os

# List files in the current directory
print(os.listdir('/content/drive/MyDrive/709/Ass2'))

['Task 1.ipynb', 'gold.csv', 'Task3 test 1.ipynb', 'Untitled0.ipynb', 'models', 'spark-warehouse', ' Task 2.ipynb', 'Task 3.ipynb', 'gold1.csv', 'gold1_single.csv']


In [41]:
!apt-get update
!apt-get install openjdk-8-jdk
!apt-get install hadoop


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,304 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,604 kB]
Get:13 http://archive.ubuntu.com/ubuntu ja

In [42]:
!pip install pyspark


In [43]:
import os
from pyspark.sql import SparkSession

# Set up Hadoop configuration
os.environ["HADOOP_CONF_DIR"] = "/path/to/hadoop/configs"

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Hadoop Integration") \
    .config("spark.hadoop.fs.defaultFS", "http://localhost:50070") \
    .getOrCreate()
